In [ ]:
using DifferentialEquations; using Plots; using WebIO; using Interact;

In [ ]:
using Reduce
ẍ = :(2*ẏ + x - (1 - μ) * (x+μ) / ((x+μ)^2 + y^2 +z^2)^(3/2) - μ * (x-1+μ) / ((x-1+μ)^2 + y^2 + z^2)^(3/2))
ÿ = :(-2*ẋ + y - (1 - μ) * (y) / ((x+μ)^2 + y^2 +z^2)^(3/2) - μ * y / ((x-1+μ)^2 + y^2 + z^2)^(3/2))
z̈ = :( -(1 - μ) * z / ((x+μ)^2 + y^2 +z^2)^(3/2) - μ * z / ((x-1+μ)^2 + y^2 + z^2)^(3/2))

In [ ]:
A =[Algebra.df(j, i) for j=[:ẋ :ẏ :ż ẍ ÿ z̈], i = [:x :y :z :ẋ :ẏ :ż]] |> x -> reshape(x, 6, 6)
B = A |> x -> reshape(x, 36) .|> (x -> Expr(:function, :((μ, x, y, z, ẋ, ẏ, ż)), x)) .|> eval |> x -> reshape(x, 6, 6)

In [ ]:
plotly();

In [698]:
A

6×6 Array{Any,2}:
 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
function U(x, xMoon, xEarth, μ)
    return 0.5*(x[1]^2+x[2]^2)+(1-μ)/norm(x - xEarth)+μ/norm(x - xMoon)
end

In [ ]:
function Udiff(xMoon, xEarth, μ)
    real = x -> U(x, xMoon, xEarth, μ); 
    return real'
end

In [697]:
using LinearAlgebra
Bline[]

36-element Array{Function,1}:
 var"#149#150"()
 var"#151#152"()
 var"#153#154"()
 var"#155#156"()
 var"#157#158"()
 var"#159#160"()
 var"#161#162"()
 var"#163#164"()
 var"#165#166"()
 var"#167#168"()
 var"#169#170"()
 var"#171#172"()
 var"#173#174"()
 ⋮
 var"#197#198"()
 var"#199#200"()
 var"#201#202"()
 var"#203#204"()
 var"#205#206"()
 var"#207#208"()
 var"#209#210"()
 var"#211#212"()
 var"#213#214"()
 var"#215#216"()
 var"#217#218"()
 var"#219#220"()

In [702]:
Bline = reshape(B, length(B));

function F(X, μ) 
    return Bline .|> (x -> x(μ, X[1], X[2], X[3], X[4], X[5], X[6])) |> x -> reshape(x, 6, 6)
end
function Xdotdot(X, μ) 
    x, y, z, ẋ, ẏ, ż = X
    ẍ = 2*ẏ + x - (1 - μ) * (x+μ) / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * (x-1+μ) / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) )
    ÿ = -2*ẋ + y - (1 - μ) * (y) / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * y / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    z̈ = - (1 - μ) * z / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * z / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    return [ẍ, ÿ, z̈]
end
function Body3!(du,u,p,t)
    z1, z, y1, y, x1, x = u;
    dz1, dz, dy1, dy, dx1, dx = du;
    func = p[1];
    dudx, dudy, dudz = func([x, y, z]);
    du[1] = dz1 = dudz;
    du[2] = dz = z1;
    du[3] = dy1 = dudy - 2*x1;
    du[4] = dy = y1;
    du[5] = dx1 = dudx + 2*y1;
    du[6] = dx = x1;
end
function Body23!(du,u,p,t)
    z1, z, y1, y, x1, x = u;
    ż1, ż, ẏ1, ẏ, ẋ1, ẋ = du;
    μ = p[1]
    du[5] = ż1 = 2*ẏ + x - (1 - μ) * (x+μ) / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * (x-1+μ) / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    du[2] = ż = z1;
    du[3] = ẏ1 = -2*ẋ + y - (1 - μ) * (y) / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * y / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    du[4] = ẏ = y1;
    du[1] = ẋ1 = - (1 - μ) * z / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * z / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    du[6] = ẋ = x1;
end
function Body33!(du,u,p,t)
    x, y, z, x1, y1, z1 = u;
    P = reshape(u[7:end], 6, 6)
    ẋ, ẏ, ż, ẋ1, ẏ1, ż1, Ṗ = du;
    μ = p[1]
    du[1] = x1;
    du[2] = y1;
    du[3] = z1;
    du[4] = 2*ẏ + x - (1 - μ) * (x+μ) / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * (x-1+μ) / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    du[5] = -2*ẋ + y - (1 - μ) * (y) / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * y / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    du[6] = - (1 - μ) * z / (( (x+μ)^2 + y^2 +z^2 )^(3/2)) - μ * z / (( (x-1+μ)^2 + y^2 + z^2 )^(3/2) );
    du[7:end] = reshape(F(u, μ) * P, 36)
end
function Body2!(du, u, p, t)
    c = p[1]
    x, x1 = u;
    ẋ, ẋ1 = du;
    if (x == 0.0)
        du[1] = 0.0;
        du[2] = 0.0;
        return;
    end
    du[1] = x1
    du[2] = -c / x^2
end
U(x, μ) = 0.5 * (x[1]^2 + x[2]^2) + (1 - μ) / norm(x + [μ, 0, 0]) + μ / norm(x + [μ - 1, 0, 0])
J(x, ẋ, μ) = 2 * U(x, μ) - norm(ẋ)^2

J (generic function with 1 method)

In [ ]:
@manipulate for xp=1.001:0.0001:1.01, yp=0.95:0.0001:1.01, zp=1.001:0.0001:1.1
    z1, z, y1, y, x1, x = u0 = [0.0,0.0073933451299313*zp,0.00042001959430719383188711*yp,0.0,0.0,1.14901111*xp]
    tspan = (0.0,0.3)
    p = [1.0 / 82.2800178]
    prob = ODEProblem(Body23!,u0,tspan,p)
    solution = solve(prob, precision=10e-17, dt=10e-16)
    # plot(solution, vars=(6, 4, 2))
    display(J([x, y, z], [x1, y1, z1], p[1]));
    plot(solution, vars=(6, 4, 2))
end

In [ ]:
@manipulate for yd0=1.0:0.1:2.0 #yd0=0.1:0.001:1, x0=1.01:0.01:1.2
    z1, z, y1, y, x1, x = u0 = [-yd0+0.000001,0.0,0.0,0.0,0.0,-1.0]
    tspan = (0.0,13.2)
    p = [ 0.0 ]#Float64(1 / 82.2800178)]
    prob = ODEProblem(Body23!,u0,tspan,p)
    solution = solve(prob)
    plot(solution, vars=(6, 4, 2))
#     display(plot(solution, vars=(0, 2)))
end

In [ ]:
using Reduce, ReduceLinAlg, ReduceAlgebra;
Usym = :(0.5*(x^2+y^2)+(1-μ)/sqrt((x)^2+(y)^2+(z)^2)+μ/sqrt((x-1)^2+(y)^2+(z)^2))

In [ ]:
a = ReduceLinAlg.hessian(Usym, (:x, :y, :z)) |> Reduce.mat;

In [ ]:
F = [[:0, :0, :0, :1, :0, :0],
     [:0, :0, :0, :0, :1, :0],
     [:0, :0, :0, :0, :0, :1],
     [a[1, 1], a[1, 2], a[1, 3], :0, :2, 0],
     [a[2, 1], a[2, 2], a[2, 3], :(-2), :0, 0],
     [a[3, 1], a[3, 2], a[3, 3], :0, :0, 0]] |> x -> reshape(x, 6)

In [ ]:
@manipulate for c=0.0:000.1:1.0 #yd0=0.1:0.001:1, x0=1.01:0.01:1.2
    ẋ, x = u0 = [1.0,0.0]
    tspan = (0.0,1.2)
    p = [ c/1.0 ]#Float64(1 / 82.2800178)]
    prob = ODEProblem(Body2!,u0,tspan,p)
    solution = solve(prob)
    plot(solution, vars=(0, 1))
end

In [ ]:
function Analitical3Body(λ, Ay, k, t)
    x = -k * Ay * cos(λ * t);
    y = Ay * sin(λ * t);
    z = 0;
    return (x, y, z)
end

@manipulate for λ = 4:0.001:6, Ay = 0.01:0.0001:0.2, k = 0.01:0.0001:1
    plot([Analitical3Body(λ, Ay, k, t) for t = 0:0.001:(π/λ) ])
end

In [ ]:
function AnaBody3!(du,u,p,t)
    x1, x, y1, y, z1, z = u;
    ẋ1, ẋ, ẏ1, ẏ, ż1, ż = du;
    c = p[1]
    du[1] = 2 * ẏ + (1 + 2*c)*x; 
    du[2] = x1;
    du[3] = -2 * ẋ -(c - 1)y
    du[4] = y1;
    du[5] = -c * z;
    du[6] = z1;
end
@manipulate for c=58.1:0.01:59 #yd0=0.1:0.001:1, x0=1.01:0.01:1.2
    ẋ, x, ẏ, y, ż, z = u0 = [0.0, -0.01, 0.0001, 0.0, 0.0, 0.0]
    tspan = (0.0,4*π)
    ok = false
    while(!ok) 
        ok = true;
        p = [ c/100.0 ]#Float64(1 / 82.2800178)]
        prob = ODEProblem(AnaBody3!,u0,tspan,p)
        solution = solve(prob)
        display(solution[end])
        if (solution[end][2] > 10.0)
            c *= 0.999999;
            ok = false;
        end
        if (solution[end][2] < -10.0)
            c *= 1.000001;
            ok = true;
        end
        display(("c" => c,"xt" => solution[end][2]))
    end
    plot(solution, vars=(2, 4, 6))
end

In [ ]:
tst = Matrix{Float64}(I, 6, 6)
tst[1:2] = [1;2]
display(tst)

In [ ]:
i = Matrix{Int}(I, 6, 6)
x, y, z, x1, y1, z1 = u0 = [1.14901111, 0.0, 0.1073933451299313, 0.0, 0.00042001959430719383188711, 0.0, reshape(i, 36)...]
tspan = (0.0, 0.3)
p = [1.0 / 82.2800178]
prob = ODEProblem(Body33!,u0,tspan,p)
solution = solve(prob, precision=10e-17, dt=10e-16)
# plot(solution, vars=(6, 4, 2))
display(J([x, y, z], [x1, y1, z1], p[1]));
plot(solution, vars=(1, 2, 3))

In [ ]:
solution[end][1:6]

In [712]:
function solveInitial(u0, Tt)
    x, y, z, x1, y1, z1 = u0
    tspan = (0.0, Tt)
    p = [1.0 / 82.2800178]
    latY0 = 0;
    display(u0[1:6]);
    for k=1:100
        
        prob = ODEProblem(Body33!,u0,tspan,p)
        solution = solve(prob, precision=10e-17, dt=10e-16)
        a = [solution(Tt)[4:6]..., Xdotdot(solution(Tt)[1:6], 1.0 / 82.2800178)...]
        phi = reshape(solution(Tt)[7:end], 6, 6);
        l = reshape(([[(phi[4, 3] - phi[2, 3] * a[4] / a[2]), (phi[4, 5] - phi[2, 5]*a[4]/a[2])]
                       [(phi[6, 3] - phi[2, 3] * a[6] / a[2]), (phi[6, 5] - phi[2, 5]*a[6]/a[2])]]), 2, 2);
        r = [a[1], a[3]]
        bump = transpose(l)^-1*r
        x, y, z, x1, y1, z1 = u0 = [x, 0.0, z + bump[1] *0.5, 0.0, y1 + bump[2]*0.5, 0.0, reshape(Matrix{Float64}(I, 6, 6), 36)...];
        display(bump)
    end
    display(u0[1:6]);
    tspan = (0.0, Tt*4.0)
    prob = ODEProblem(Body33!,u0,tspan,p)
    solution = solve(prob, precision=10e-17, dt=10e-16)
    return solution;
end
# function solveInitial2(u0)
#     x, y, z, x1, y1, z1 = u0
#     tspan = (0.0, 0.01)
#     p = [1.0 / 82.2800178]
#     solution = 0;
#     for k=1:100
#         prob = ODEProblem(Body33!,u0,tspan,p)
#         solution = solve(prob, precision=10e-17, dt=10e-16)
#         a = [solution[end][4:6]..., Xdotdot(solution[end][1:6], 1.0 / 82.2800178)...]
#         phi = reshape(solution[end][7:end], 6, 6)
#         dt2 = -phi[2, 1]*
#         r = [-a[1], -a[3]]
#         bump = l*r
#         x, y, z, x1, y1, z1 = u0 = [x, 0.0, z + bump[1], 0.0, y1 - bump[2], 0.0, reshape(i, 36)...];
#         if (k == 99)
#             display(bump);
#             tspan = (0.0, 4.0)
#             prob = ODEProblem(Body33!,u0,tspan,p)
#             solution = solve(prob, precision=10e-17, dt=10e-16)
#             return solution;
#         end
#     end
# end


In [713]:
i = Matrix{Float64}(I, 6, 6);
plot([1, 2, 3])
solved = solveInitial([1.36111, 0.0, 0.001, 0.0, 0.00019430719383188711, 0.0, reshape(i, 36)...], 0.35)
plot(solved, vars=(0, 4))
plot!(solved, vars=(0, 2))
plot!(solved, vars=(0, 6))

6-element Array{Float64,1}:
 1.36111
 0.0
 0.001
 0.0
 0.0001943071938318871
 0.0

2-element Array{Float64,1}:
 0.0002306674070414864
 0.05389789081180874

2-element Array{Float64,1}:
 0.00020385681483612987
 0.032536542480484126

2-element Array{Float64,1}:
 0.00018793877837905643
 0.018334599172025203

2-element Array{Float64,1}:
 0.0001816441023384601
 0.00985855909600436

...

2-element Array{Float64,1}:
 0.03414238843788464
 0.0005651891191158741

2-element Array{Float64,1}:
 0.03560772310148741
 0.0006001687629173524

2-element Array{Float64,1}:
 0.037113757296105776
 0.0006417366234481286

6-element Array{Float64,1}:
 1.36111
 0.0
 0.3667849844625114
 0.0
 0.07059562827559449
 0.0

<!DOCTYPE html>
 
 
 Plots.jl

In [690]:
plot(solved, vars=(1,2,3))

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
display(F(solved[1][1:6], 0.01))
# display(reshape(solved[1][7:end]))
display(J(solved[3][1:3], solved[3][4:6], 1.0 / 82.2800178))
display(J(solved[10][1:3], solved[10][4:6], 1.0 / 82.2800178))

In [724]:
function minY(sol, interval)
    curMin = (10e20, 0)
    for i=interval
        if (abs(sol(i)[2]) < curMin[1])
            curMin = (i, sol(i))
        end
    end
    return curMin;
end

minY (generic function with 1 method)

In [726]:
minY(solved, 0.0001:0.00001:0.06)

(0.06, [1.362907739886677, 0.004225140262357338, 0.36650212089954876, 0.05995332205253183, 0.07000337579977321, -0.009421177712521376, 1.002992598762972, -0.00011709317620902832, 0.0006916631935028228, 0.09967356805074325  …  4.541703177069923e-7, 0.11987823152447616, 0.9938326051535791, 3.0237848218996227e-5, 1.381128919203611e-5, -3.7458090928058795e-7, 0.05999153504002231, 0.0006895986525647632, -2.495975580785587e-5, 0.9995767006964716])